In [ ]:
import os
import glob
from pathlib import Path

# YOLO 포맷 데이터셋 저장 경로
dataset_root = "archive"
output_dir = "yolo_dataset"
os.makedirs(output_dir, exist_ok=True)

# YOLO 데이터셋 구조 생성
for split in ["train", "test", "validation"]:
    os.makedirs(f"{output_dir}/images/{split}", exist_ok=True)
    os.makedirs(f"{output_dir}/labels/{split}", exist_ok=True)

# 클래스 목록 자동 탐색
class_names = sorted(next(os.walk(f"{dataset_root}/train"))[1])  # train 폴더의 하위 폴더가 클래스

# 클래스 이름을 ID로 매핑
class_to_id = {name: idx for idx, name in enumerate(class_names)}
print("클래스 매핑:", class_to_id)

# 이미지 및 라벨 변환 함수
def convert_to_yolo_format(image_path, class_id, output_label_dir):
    label_file = Path(output_label_dir) / f"{Path(image_path).stem}.txt"
    with open(label_file, "w") as f:
        # 현재는 Bounding Box 정보가 없으므로, 임시로 전체 이미지(0 0.5 0.5 1.0 1.0)로 설정
        f.write(f"{class_id} 0.5 0.5 1.0 1.0\n")

# 데이터 변환 실행
for split in ["train", "test", "validation"]:
    split_path = os.path.join(dataset_root, split)

    for class_name in class_names:
        class_id = class_to_id[class_name]
        image_paths = glob.glob(f"{split_path}/{class_name}/*.jpg")

        for img_path in image_paths:
            # 이미지 복사
            img_output_path = f"{output_dir}/images/{split}/{Path(img_path).name}"
            os.system(f"cp {img_path} {img_output_path}")

            # YOLO 라벨 생성
            convert_to_yolo_format(img_path, class_id, f"{output_dir}/labels/{split}")

print("✅ 데이터 변환 완료! YOLO 형식으로 저장됨.")


In [ ]:
# dataset.yaml 생성
dataset_yaml = f"{output_dir}/dataset.yaml"

with open(dataset_yaml, "w") as f:
    f.write(f"train: {output_dir}/images/train\n")
    f.write(f"val: {output_dir}/images/validation\n")
    f.write(f"test: {output_dir}/images/test\n\n")
    f.write(f"nc: {len(class_names)}\n")
    f.write(f"names: {class_names}\n")

print(f"✅ {dataset_yaml} 생성 완료!")
